<a href="https://colab.research.google.com/github/Prabha14039/landslide-prediction/blob/main/main2_(without_gen_algo).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycaret

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Load the dataset
dataset = pd.read_csv('Book2new.csv')
feature_types = dataset.dtypes

# Identify categorical features
categorical_features = feature_types[feature_types == 'object'].index.tolist()

# Print the list of categorical features
print("Categorical Features:", categorical_features)

# Identify numerical features
numerical_features = feature_types[feature_types != 'object'].index.tolist()

# Print the list of numerical features
print("Numerical Features:", numerical_features)


Categorical Features: ['State', 'District', 'Toposheet', 'Material Type', 'Movement Type', 'Failure Plane', 'Activity', 'Distribution', 'Style', 'Failure Mechanism', 'Geomorphology', 'Hydrology', 'Susceptibility', 'Risk', 'Triggering Factor', 'Geoscientific Causes']
Numerical Features: ['Longitude', 'Latitude', 'Length', 'Width', 'Height']


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Load the dataset
dataset = pd.read_csv('Book2new.csv')

# Assuming 'columns_to_drop' contains the names of columns you want to drop
columns_to_drop = ['Toposheet'
                   ]

# Drop the specified columns from the DataFrame
dataset.drop(columns=columns_to_drop, inplace=True)

# Define categorical features
categorical_features = ['State', 'District', 'Material Type', 'Movement Type','Style','Distribution',
                        'Failure Plane', 'Activity', 'Failure Mechanism','Risk','Susceptibility',
                        'Geomorphology', 'Hydrology', 'Triggering Factor','Geoscientific Causes']

# Perform one-hot encoding for categorical features
one_hot_encoder = OneHotEncoder()  # No sparse parameter specified
encoded_features = one_hot_encoder.fit_transform(dataset[categorical_features])

# Convert the sparse matrix to a dense array
encoded_features_array = encoded_features.toarray()

# Get the unique category names for each categorical feature
encoded_columns = []
for i, col in enumerate(categorical_features):
    unique_values = dataset[col].unique()
    for value in unique_values:
        encoded_columns.append(f"{col}_{value}")

# Convert the encoded features array into a DataFrame
encoded_df = pd.DataFrame(encoded_features_array, columns=encoded_columns)

# Concatenate the encoded features DataFrame with any remaining numeric features
X = pd.concat([encoded_df, dataset.select_dtypes(include=[np.number]).reset_index(drop=True)], axis=1)

# Drop the 'Geoscientific Causes' column from the feature matrix (assuming it's the target variable)
y = dataset['Risk']

print(X.head())
print(y.head())


In [ ]:

X


In [6]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define preprocessing steps for numerical and categorical features
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(exclude=[np.number]).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
    ('scaler', StandardScaler())  # Standardize features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent value
    ('onehot', OneHotEncoder())  # One-hot encode categorical features
])

# Combine preprocessing steps for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocess the entire dataset
X_processed = preprocessor.fit_transform(X)

# X_processed is the preprocessed dataset


In [7]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder_y = LabelEncoder()

# Encode the target variable (y)
y_encoded = label_encoder_y.fit_transform(y)


# Remove rows with NaNs in y_encoded and corresponding rows in X
indices_with_nan = np.isnan(y_encoded)
X_processed = X[~indices_with_nan]
y_processed = y_encoded[~indices_with_nan]


In [8]:
import pandas as pd

# Assuming X is your feature matrix
# Replace NaN values with 0
X_filled = X_processed.fillna(0)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer

# Initialize LabelEncoder
label_encoder_y = LabelEncoder()

# Encode the target variable (y)
y_encoded = label_encoder_y.fit_transform(y)

# Create KNN imputer
imputer = KNNImputer(n_neighbors=2)

# Perform imputation on y_encoded
y_imputed = imputer.fit_transform(y_encoded.reshape(-1, 1))

# Convert back to original labels
y_processed = label_encoder_y.inverse_transform(y_imputed.astype(int).ravel())

# Assuming X is your feature matrix
# Perform imputation on X
X_processed = imputer.fit_transform(X)

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pycaret.regression import *
from sklearn.model_selection import train_test_split
import pandas as pd

# Define preprocessing steps for numerical and categorical features
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(exclude=[np.number]).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
    ('scaler', StandardScaler())  # Standardize features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=2)),  # Impute missing values with KNN
    ('onehot', OneHotEncoder())  # One-hot encode categorical features
])

# Combine preprocessing steps for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocess the entire dataset
X_processed = preprocessor.fit_transform(X)

# Encode the target variable (y)
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y_encoded, test_size=0.2, random_state=42)

# Create a DataFrame from X_train and y_train
train_data = pd.DataFrame(X_train, columns=X_processed.columns)
train_data['Label'] = y_train  # Rename the target column to 'Label'

# Create a DataFrame from X_test and y_test
test_data = pd.DataFrame(X_test, columns=X_processed.columns)
test_data['Label'] = y_test  # Rename the target column to 'Label'

# Initialize PyCaret
reg_setup = setup(data=train_data, target='Label')



# Compare Regression Models
best_model = compare_models()

# Evaluate the Best Model on the test set
evaluate_model(best_model)

# Make predictions on the test set
predictions = predict_model(best_model, data=test_data)

# Further steps (if needed) such as plotting, saving the model, etc.

In [9]:
from pycaret.regression import *
from sklearn.model_selection import train_test_split  # Add this import statement
import pandas as pd

# Assuming X and y are your feature matrix and target variable
X_train, X_test, y_train, y_test = train_test_split(X_filled, y_processed, test_size=0.2, random_state=0)

# Create a DataFrame from X_train and y_train
train_data = pd.DataFrame(X_train, columns=X_filled.columns)
train_data['Label'] = y_train  # Rename the target column to 'Label'

# Create a DataFrame from X_test and y_test
test_data = pd.DataFrame(X_test, columns=X_filled.columns)
test_data['Label'] = y_test  # Rename the target column to 'Label'

# Initialize PyCaret
reg_setup = setup(data=train_data, target='Label')

# Compare Regression Models
best_model = compare_models()

# Evaluate the Best Model on the test set
evaluate_model(best_model)

# Make predictions on the test set
predictions = predict_model(best_model, data=test_data)

# Further steps (if needed) such as plotting, saving the model, etc.


,Description,Value
0,Session id,6313
1,Target,Label
2,Target type,Regression
3,Original data shape,"(534, 367)"
4,Transformed data shape,"(534, 367)"
5,Transformed train set shape,"(373, 367)"
6,Transformed test set shape,"(161, 367)"
7,Numeric features,366
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,0.0053,0.0105,0.0324,0.9935,0.0044,0.0009,0.0770
et,Extra Trees Regressor,0.0179,0.0410,0.1066,0.9758,0.0160,0.0027,0.3380
gbr,Gradient Boosting Regressor,0.0237,0.0439,0.1151,0.9738,0.0208,0.0054,0.2540
lr,Linear Regression,0.0448,0.0497,0.1687,0.9681,0.0336,0.0127,0.5820
br,Bayesian Ridge,0.0448,0.0497,0.1687,0.9681,0.0336,0.0127,0.1270
dt,Decision Tree Regressor,0.0187,0.0669,0.1144,0.9607,0.0190,0.0027,0.0790
rf,Random Forest Regressor,0.0280,0.0765,0.1635,0.9554,0.0260,0.0047,0.4360
ridge,Ridge Regression,0.1479,0.1440,0.3206,0.9091,0.1000,0.0513,0.0800
ada,AdaBoost Regressor,0.2044,0.1814,0.3901,0.8787,0.0862,0.0741,0.2080
lightgbm,Light Gradient Boosting Machine,0.3546,0.5609,0.7124,0.6118,0.2627,0.1019,0.2560


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Orthogonal Matching Pursuit,0.0075,0.0075,0.0864,0.9960,0.0116,0.0012


In [ ]:
from pycaret.regression import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd

# Assuming X and y are your feature matrix and target variable
# Perform genetic algorithm optimization
# Define a fitness function
def fitness_function(chromosome, X, y):
    selected_features = X[:, chromosome]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(selected_features)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return r2_score(y_test, y_pred)

# Genetic Algorithm Parameters
population_size = 100
chromosome_length = X_filled.shape[1]
mutation_rate = 0.1
num_generations = 50

# Initialize population
population = np.random.randint(2, size=(population_size, chromosome_length))

# Genetic Algorithm Main Loop
for generation in range(num_generations):
    # Evaluate fitness of each individual in the population
    fitness_scores = np.array([fitness_function(chromosome, X_filled, y_processed) for chromosome in population])

    # Select parents for reproduction (tournament selection)
    selected_indices = np.random.choice(population_size, size=population_size, replace=True)
    selected_population = population[selected_indices]

    # Crossover
    crossover_point = np.random.randint(1, chromosome_length, size=population_size)
    offspring_population = np.empty_like(selected_population)
    for i in range(population_size):
        parent1, parent2 = selected_population[i], selected_population[(i+1) % population_size]
        offspring_population[i] = np.concatenate((parent1[:crossover_point[i]], parent2[crossover_point[i]:]))

    # Mutation
    mutation_mask = np.random.random(offspring_population.shape) < mutation_rate
    offspring_population[mutation_mask] = 1 - offspring_population[mutation_mask]

    # Replace the old population with the offspring population
    population = offspring_population

# Find the best solution (chromosome) in the final population
best_chromosome = population[np.argmax(fitness_scores)]

# Extract the selected features based on the best chromosome
selected_features = X_filled[:, best_chromosome.astype(bool)]

# Combine selected features with original dataset
X_train_selected = np.hstack((X_filled, selected_features))

# Convert to DataFrame
train_data = pd.DataFrame(X_train_selected, columns=list(X_filled.columns) + ['Selected_Features'])
train_data['Label'] = y_processed

# Initialize PyCaret
reg_setup = setup(data=train_data, target='Label')

# Compare Regression Models
best_model = compare_models()

# Evaluate the Best Model on the test set
evaluate_model(best_model)

# Make predictions on the test set
predictions = predict_model(best_model, data=test_data)

# Further steps (if needed) such as plotting, saving the model, etc.
